In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, get_linear_schedule_with_warmup, TrainingArguments, BertModel, AdamW
from datasets import load_dataset
import numpy as np
from torch.utils.data import DataLoader
import tqdm
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [104]:
class Config(object):

    """配置参数"""
    def __init__(self, dataset):
        self.model_name = 'bert'
        self.train_path = dataset + '/homework/train.txt'                                # 训练集                                  # 验证集
        self.test_path = dataset + '/homework/test.txt'                                  # 测试集
        self.class_list = [x.strip() for x in open(
            dataset + '/homework/class.txt').readlines()]                                # 类别名单
        self.save_path = dataset + '/saved_dict/' + self.model_name       # 模型训练结果
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')   # 设备

        self.require_improvement = 1000                                 # 若超过1000batch效果还没提升，则提前结束训练
        self.num_classes = len(self.class_list)                         # 类别数
        self.num_epochs = 50                                        # epoch数
        self.batch_size = 32                                           # mini-batch大小
        self.pad_size = 64                                              # 每句话处理成的长度(短填长切)
        self.learning_rate = 5e-5                                       # 学习率
        self.bert_path = '/opt/data/private/huggingface/models--bert-base-chinese/snapshots/c30a6ed22ab4564dc1e3b2ecbf6e766b0611a33f'
        self.tokenizer = BertTokenizer.from_pretrained(self.bert_path)
        self.hidden_size = 768

In [105]:
class BModel(nn.Module):

    def __init__(self, config):
        super(BModel, self).__init__()
        self.bert = BertModel.from_pretrained(config.bert_path)
        for param in self.bert.parameters():
            param.requires_grad = True
        self.fc = nn.Linear(config.hidden_size, config.num_classes)
        self.sigmod = nn.Sigmoid()

    def forward(self, x):
        context = x[0]  # 输入的句子
        mask = x[2]  # 对padding部分进行mask，和句子一个size，padding部分用0表示，如：[1, 1, 1, 1, 0, 0]
        out = self.bert(context, attention_mask=mask)
        out = out.last_hidden_state[:, 0]
        out = self.fc(out)
        out = self.sigmod(out)
        return out

In [107]:
class DatasetIterater(object):
    def __init__(self, batches, batch_size, device):
        self.batch_size = batch_size
        self.batches = batches
        self.n_batches = len(batches) // batch_size
        self.residue = False  # 记录batch数量是否为整数
        if len(batches) % self.n_batches != 0:
            self.residue = True
        self.index = 0
        self.device = device

    def _to_tensor(self, datas):
        x = torch.LongTensor([_[0] for _ in datas]).to(self.device)
        y = torch.LongTensor([_[1] for _ in datas]).to(self.device)

        # pad前的长度(超过pad_size的设为pad_size)
        seq_len = torch.LongTensor([_[2] for _ in datas]).to(self.device)
        mask = torch.LongTensor([_[3] for _ in datas]).to(self.device)
        return (x, seq_len, mask), y

    def __next__(self):
        if self.residue and self.index == self.n_batches:
            batches = self.batches[self.index * self.batch_size: len(self.batches)]
            self.index += 1
            batches = self._to_tensor(batches)
            return batches

        elif self.index >= self.n_batches:
            self.index = 0
            raise StopIteration
        else:
            batches = self.batches[self.index * self.batch_size: (self.index + 1) * self.batch_size]
            self.index += 1
            batches = self._to_tensor(batches)
            return batches

    def __iter__(self):
        return self

    def __len__(self):
        if self.residue:
            return self.n_batches + 1
        else:
            return self.n_batches

In [108]:
PAD, CLS = '[PAD]', '[CLS]'
def build_dataset(config):
    def load_dataset(path, pad_size=64):
        contents = []
        with open(path, 'r', encoding='UTF-8') as f:
            for line in f:
                lin = line.strip()
                if not lin:
                    continue
                content = lin.split('\t')[0]
                label_list = [w.split('#')[0] for w in lin.split('\t')[1:]]
                label = [0] * config.num_classes
                for l in label_list:
                    label[config.class_list.index(l)] = 1
                token = config.tokenizer.tokenize(content)
                token = [CLS] + token
                seq_len = len(token)
                mask = []
                token_ids = config.tokenizer.convert_tokens_to_ids(token)

                if pad_size:
                    if len(token) < pad_size:
                        mask = [1] * len(token_ids) + [0] * (pad_size - len(token))
                        token_ids += ([0] * (pad_size - len(token)))
                    else:
                        mask = [1] * pad_size
                        token_ids = token_ids[:pad_size]
                        seq_len = pad_size
                contents.append((token_ids, label, seq_len, mask))
        return contents
    train = load_dataset(config.train_path, config.pad_size)
    test = load_dataset(config.test_path, config.pad_size)
    return train, test

In [109]:
dataset = 'data'
config = Config(dataset)
train_data, test_data = build_dataset(config)
train_iter = DatasetIterater(train_data, config.batch_size, config.device)
test_iter = DatasetIterater(test_data, config.batch_size, config.device)

In [110]:
def evaluate(config, model, data_iter, test=False):
    model.eval()
    loss_total = 0
    predict_all = []
    labels_all = []
    criterion = nn.BCELoss()
    with torch.no_grad():
        for texts, labels in data_iter:
            outputs = model(texts)
            loss = criterion(outputs, labels.float())
            loss_total += loss
            labels = labels.data.cpu().numpy()
            predic = (outputs.data > 0.5).int().cpu().numpy()
            assert len(labels) == len(predic)
            for i in range(len(labels)):
                labels_all.append(labels[i])
                predict_all.append(predic[i])

    acc = metrics.accuracy_score(labels_all, predict_all)
    if test:
        report = metrics.classification_report(np.array(labels_all), np.array(predict_all), target_names=config.class_list, digits=4)
        return acc, loss_total / len(data_iter), report
    return acc, loss_total / len(data_iter)

In [111]:
def train(config, model, train_iter, test_iter):
    model.train()
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]
    optimizer = AdamW(optimizer_grouped_parameters,
                         lr=config.learning_rate)
    criterion = nn.BCELoss()
    
    total_batch = 0  # 记录进行到多少batch
    flag = False  # 记录是否很久没有效果提升
    train_best_loss = float('inf')
    model.train()
    for epoch in range(config.num_epochs):
        print('Epoch [{}/{}]'.format(epoch + 1, config.num_epochs))
        for i, (trains, labels) in enumerate(train_iter):
            outputs = model(trains)
            model.zero_grad()
            loss = criterion(outputs, labels.float())
            loss.backward()
            optimizer.step()
            if total_batch % 10 == 0:
                # 每多少轮输出在训练集和验证集上的效果
                true = labels.data.cpu()
                predic = (outputs.data > 0.5).int().cpu()
                train_acc = metrics.accuracy_score(true, predic)
                msg = 'Iter: {0:>3},    Train Loss: {1:>3.2},     Train Acc: {2:>3.2%}'
                print(msg.format(total_batch, loss.item(), train_acc))
            total_batch += 1
        torch.save(model.state_dict(), config.save_path+ '_{}.ckpt'.format(epoch))

In [112]:
def test(config, model, test_iter):
    # test
    model.load_state_dict(torch.load(config.save_path + '_{}.ckpt'.format(config.num_epochs - 1)))
    model.eval()
    test_acc, test_loss, test_report = evaluate(config, model, test_iter, test=True)
    msg = 'Test Loss: {0:>3.2},  Test Acc: {1:>3.2%}'
    print(msg.format(test_loss, test_acc))
    print("Precision, Recall and F1-Score...")
    print(test_report)

In [79]:
model = BModel(config).to(config.device)
train(config, model, train_iter, test_iter)

Some weights of the model checkpoint at /opt/data/private/huggingface/models--bert-base-chinese/snapshots/c30a6ed22ab4564dc1e3b2ecbf6e766b0611a33f were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/root/miniconda3/lib/python3.9/site-packages/transformers/optimiza

Epoch [1/50]
Iter:   0,    Train Loss: 0.68,     Train Acc: 0.00%
Iter:  10,    Train Loss: 0.33,     Train Acc: 0.00%
Iter:  20,    Train Loss: 0.32,     Train Acc: 15.62%
Iter:  30,    Train Loss: 0.5,     Train Acc: 9.38%
Iter:  40,    Train Loss: 0.22,     Train Acc: 53.12%
Iter:  50,    Train Loss: 0.25,     Train Acc: 18.75%
Iter:  60,    Train Loss: 0.23,     Train Acc: 46.88%
Iter:  70,    Train Loss: 0.12,     Train Acc: 81.25%
Epoch [2/50]
Iter:  80,    Train Loss: 0.11,     Train Acc: 78.12%
Iter:  90,    Train Loss: 0.2,     Train Acc: 65.62%
Iter: 100,    Train Loss: 0.16,     Train Acc: 68.75%
Iter: 110,    Train Loss: 0.1,     Train Acc: 84.38%
Iter: 120,    Train Loss: 0.19,     Train Acc: 56.25%
Iter: 130,    Train Loss: 0.43,     Train Acc: 12.50%
Iter: 140,    Train Loss: 0.16,     Train Acc: 68.75%
Iter: 150,    Train Loss: 0.077,     Train Acc: 90.62%
Iter: 160,    Train Loss: 0.13,     Train Acc: 68.75%
Iter: 170,    Train Loss: 0.12,     Train Acc: 68.75%
Iter: 1

In [113]:
test(config, model, test_iter)

Test Loss: 0.17,  Test Acc: 80.66%
Precision, Recall and F1-Score...
              precision    recall  f1-score   support

          动力     0.8698    0.9556    0.9107       811
          价格     0.9645    0.8892    0.9253       397
          内饰     0.8208    0.9221    0.8685       154
          配置     0.8577    0.8707    0.8642       263
         安全性     0.7216    0.9246    0.8106       199
          外观     0.8175    0.8819    0.8485       127
          操控     0.8431    0.8165    0.8296       316
          油耗     0.9320    0.9375    0.9347       336
          空间     0.9043    0.8333    0.8673       102
         舒适性     0.7105    0.8120    0.7579       266

   micro avg     0.8524    0.8984    0.8748      2971
   macro avg     0.8442    0.8843    0.8617      2971
weighted avg     0.8578    0.8984    0.8759      2971
 samples avg     0.8817    0.9086    0.8856      2971



/root/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
def build_ADBSdataset(config):
    def load_dataset(path, pad_size=64):
        contents = []
        with open(path, 'r', encoding='UTF-8') as f:
            for line in f:
                lin = line.strip()
                if not lin:
                    continue
                content = lin.split('\t')[0]
                aspect_list = [w.split('#')[0] for w in lin.split('\t')[1:]]
                label = [int(w.split('#')[1]) + 1 for w in lin.split('\t')[1:]]
                for i in range(len(aspect_list)): 
                    token = config.tokenizer.tokenize(aspect_list[i] + content)
                    token = [CLS] + token
                    seq_len = len(token)
                    mask = []
                    token_ids = config.tokenizer.convert_tokens_to_ids(token)

                    if pad_size:
                        if len(token) < pad_size:
                            mask = [1] * len(token_ids) + [0] * (pad_size - len(token))
                            token_ids += ([0] * (pad_size - len(token)))
                        else:
                            mask = [1] * pad_size
                            token_ids = token_ids[:pad_size]
                            seq_len = pad_size
                    contents.append([token_ids, np.eye(3)[label[i]], seq_len, mask])
        return contents
    train = load_dataset(config.train_path, config.pad_size)
    test = load_dataset(config.test_path, config.pad_size)
    return train, test

In [165]:
train_ADBSdata, test_ADBSdata = build_ADBSdataset(config)

In [166]:
from torch.utils.data import Dataset, DataLoader
class ABSADataset(Dataset):
    def __init__(self, data, device):
        super(ABSADataset, self).__init__()
        self.dataset = data
        self.device = device

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        text = torch.LongTensor(self.dataset[idx][0]).to(self.device)
        label = torch.LongTensor([self.dataset[idx][1]]).to(self.device)
        seq_len = torch.LongTensor([self.dataset[idx][2]]).to(self.device)
        mask = torch.LongTensor(self.dataset[idx][3]).to(self.device)

        return text, seq_len, mask, label

In [167]:
train_ADBSdataset = ABSADataset(train_ADBSdata, config.device)
train_loader = DataLoader(train_ADBSdataset, batch_size=config.batch_size, shuffle=True)
test_ADBSdataset = ABSADataset(test_ADBSdata, config.device)
test_loader = DataLoader(test_ADBSdataset, batch_size=config.batch_size, shuffle=True)

In [168]:
model = BertForSequenceClassification.from_pretrained(config.bert_path, num_labels=1)
model.to(config.device)

Some weights of the model checkpoint at /opt/data/private/huggingface/models--bert-base-chinese/snapshots/c30a6ed22ab4564dc1e3b2ecbf6e766b0611a33f were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some w

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [169]:

optimizer = AdamW(model.parameters(), lr=5e-5)
total_steps = len(train_loader) * config.num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
print(len(train_loader))

301


/root/miniconda3/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
for epoch in range(1):
    model.train()
    for step, (input_ids, seq_len, attention_mask, labels) in enumerate(train_loader):
        input_ids = input_ids
        attention_mask = attention_mask
        labels = labels.float()
        
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        if step % 10 == 0:
            print(f'Epoch: {epoch}, Step: {step}, Loss: {loss.item()}')

Epoch: 0, Step: 0, Loss: 1.0637214183807373
Epoch: 0, Step: 10, Loss: 0.39778560400009155
Epoch: 0, Step: 20, Loss: 0.3881106376647949
Epoch: 0, Step: 30, Loss: 0.3352685570716858
Epoch: 0, Step: 40, Loss: 0.3070780634880066
Epoch: 0, Step: 50, Loss: 0.3166796863079071
Epoch: 0, Step: 60, Loss: 0.4969061017036438
Epoch: 0, Step: 70, Loss: 0.22093801200389862
Epoch: 0, Step: 80, Loss: 0.471129834651947
Epoch: 0, Step: 90, Loss: 0.22954508662223816
Epoch: 0, Step: 100, Loss: 0.3182803988456726
Epoch: 0, Step: 110, Loss: 0.24896395206451416
Epoch: 0, Step: 120, Loss: 0.32637643814086914
Epoch: 0, Step: 130, Loss: 0.3014448881149292
Epoch: 0, Step: 140, Loss: 0.3703632354736328
Epoch: 0, Step: 150, Loss: 0.44550827145576477
Epoch: 0, Step: 160, Loss: 0.2635398209095001
Epoch: 0, Step: 170, Loss: 0.3031829297542572
Epoch: 0, Step: 180, Loss: 0.3802475929260254
Epoch: 0, Step: 190, Loss: 0.23148801922798157
Epoch: 0, Step: 200, Loss: 0.27418023347854614
Epoch: 0, Step: 210, Loss: 0.249417603

In [ ]:
all_predictions = []
all_labels = []

with torch.no_grad():
    for step, (input_ids, seq_len, attention_mask, labels) in enumerate(test_loader):
        input_ids = input_ids
        attention_mask = attention_mask
        labels = labels.float()

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # 将预测结果从张量转换为列表
        predictions = logits.squeeze(-1).cpu().numpy().tolist()
        labels = labels.cpu().numpy().tolist()
        print(len(predictions[0]))
        all_predictions.extend(predictions)
        all_labels.extend(labels)
print(all_predictions)
# report = metrics.classification_report(all_labels, all_predictions, target_names=['负向', '中性', "正向"])
# print("Precision, Recall and F1-Score...")
# print(report)

TypeError: object of type 'float' has no len()